In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import codecs
from collections import Counter

In [2]:
#ssh -L 9200:127.0.0.1:9200 claudiao@vision.ucd.ie
#ssh -L 5601:127.0.0.1:5601 claudiao@vision.ucd.ie

#####################################################################
#PARA INDEXAR DATA EN ELASTICSEARCH
#1- Compress data to be indexed: cat *.json | gzip > name.gz
#2- Create index: Change to the elasticsearch-2.x.x/elasticsearch/ directory, there type ./elasticsearch.sh -c indexName
#3- Index data: ./elasticsearch.sh -l /pathToCompressedFile indexName
#4- Delete index: ./elasticsearch.sh -d indexName
#####################################################################

dirname = '/home/clau/Documents/journalists2017_personas/'
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 50}])

journos_ids = list()
tweet_ids = set()
number_tweets = Counter()
user_tweets = dict()

#journos_file = codecs.open(dirname+'uk/personalAccounts_423.txt', 'r')
journos_file = codecs.open(dirname+'ireland_ids.txt', 'r')
for line in journos_file:
    journos_ids.append(line.strip())
journos_file.close()

#response = scan(es, index='uk', q='*', scroll='3m')
response = scan(es, index='ireland2017', q='*', scroll='3m')

for hit in response:
    if hit['_source']['id'] not in tweet_ids: #and str(hit['_source']['user']['id']) not in journos_ids:
        #Sprint('audience')
        tweet_ids.add(hit['_source']['id'])
        #if number_tweets[hit['_source']['user']['id']] <= 100:
        number_tweets[hit['_source']['user']['id']] += 1
        if hit['_source']['user']['id'] in user_tweets:
            user_tweets[hit['_source']['user']['id']] = user_tweets[hit['_source']['user']['id']]+'\t'+hit['_source']['text'].replace('\n', ' ')+'\t'+ hit['_source']['created_at']+'\n'
        else:
            user_tweets[hit['_source']['user']['id']] = hit['_source']['text'].replace('\n', ' ')+'\t'+ hit['_source']['created_at']+'\n'

GET http://localhost:9200/ireland2017/_search?size=1000&q=%2A&scroll=3m&search_type=scan [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.5/dist-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/usr/local/lib/python3.5/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/clau/.local/lib/python3.5/site-packages/elasticsearch/connection/http_urllib3.py", line 94, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py"

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f980465a898>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f980465a898>: Failed to establish a new connection: [Errno 111] Connection refused)

In [3]:
for k,v in user_tweets.items():
    if number_tweets[k] >= 20:
        user_file = codecs.open('/home/clau/Documents/journalists2017_personas/audience_tweets_ireland/'+str(k)+'.txt', 'w', encoding='utf-8')
        #user_file = codecs.open(dirname+'audience_tweets_ireland/'+str(k)+'.txt', 'w', encoding='utf-8')
        #user_file = codecs.open(dirname+'audience_tweets_uk/'+str(k)+'.txt', 'w', encoding='utf-8')
        user_file.write(str(v).replace('\n\t','\n').strip())
        user_file.close()

In [4]:
# for k,v in user_tweets.items():
#     if number_tweets[k] >= 50:
#         user_file = codecs.open(dirname+'audience_retweets_ireland/'+str(k)+'.txt', 'w', encoding='utf-8')
#         #user_file = codecs.open(dirname+'audience_tweets_uk/'+str(k)+'.txt', 'w', encoding='utf-8')
#         user_file.write(str(v).replace('\n\t','\n').strip())
#         user_file.close()

In [4]:
#counts_file = codecs.open(dirname+'audience_tweet_count_uk.txt', 'w', encoding='utf-8')
counts_file = codecs.open('/home/clau/Documents/journalists2017_personas/audience_tweet_count_ireland.txt', 'w', encoding='utf-8')
#counts_file = codecs.open(dirname+'audience_tweet_count_ireland.txt', 'w', encoding='utf-8')
for k,v in number_tweets.items():
    if number_tweets[k] >= 20:
        counts_file.write(str(k)+'\t'+str(v)+'\n')
counts_file.close()

In [5]:
#descriptions_file = codecs.open(dirname+'audience_descriptions_uk.txt', 'w', encoding='utf-8')
descriptions_file = codecs.open(dirname+'audience_descriptions_ireland.txt', 'w', encoding='utf-8')
for k,v in user_tweets.items():
    if number_tweets[k] >= 20:
        response = scan(es, index='ireland2017', q='user.id:'+str(k), scroll='3m')
        #response = scan(es, index='ireland', q='user.id:'+str(k), scroll='3m')
        for hit in response:
            if hit['_source']['user']['description'] is not None:
                descriptions_file.write(str(k)+'\t'+hit['_source']['user']['screen_name']+'\t'+hit['_source']['user']['description']+'\t'+str(hit['_source']['user']['verified'])+'\t'+hit['_source']['user']['name']+'\n')
            else:
                descriptions_file.write(str(k)+'\t'+hit['_source']['user']['screen_name']+'\t\t'+str(hit['_source']['user']['verified'])+'\t'+hit['_source']['user']['name']+'\n')
            break
descriptions_file.close()

In [5]:
# dirname = '/home/clau/Documents/journal_five_countries/'
# es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 50}])

# ids_file = codecs.open(dirname+'journos_uk_ids.txt', 'r', encoding='utf-8')
# for line in ids_file:
#         #response = scan(es, index='uk', q='user.id:'+str(k), scroll='3m')
#         response = scan(es, index='uk', q='user.id:'+line.strip(), scroll='3m')
#         for hit in response:
#             print(line.strip() +'\t' +str(hit['_source']['user']['name']))
#             break
# ids_file.close()

15374525	Adrían Bridgwater
4970411	Al Jazeera English
122941715	Aasmah Mir
20088745	Adam Gabbatt
19030517	Alexi Mostrous
21697211	Alice Bhandhukravi
97278426	Alice Tarleton
57420624	Allister Heath
16513719	Amy Willis
15778426	AndrewSparrow
47944953	ANGELA WALKER
50284910	Anna Adams
124184046	An Filip
18381609	AssedBaig
111310036	George Alagiah
125656811	Barbara Plett
143415291	BBC Breakfast
621523	BBC Business
19717082	BBC HARDtalk
204265733	James Vincent
130104942	Jon Sopel
44638576	Kim Ghattas
61183568	Laura Kuenssberg
612473	BBC News (UK)
20543416	BBC Newsnight
621533	BBC Politics
23937508	BBC Radio 4
250651231	Tim Whewell
197401382	Tim Willcox
742143	BBC News (World)
20178144	Joe Lynam BBC Biz
13390772	BBCdandamon
198118653	Ben Chu
19902709	Beth Rigby
19477552	Rebecca Williams
133356125	beyondbrics
21406106	Brian Whitaker
18055216	Steve Busfield
25070893	Guardian Business
203560532	Nick Davies
195792174	Emily Wilson
11269592	CarolLewis
22460658	Chris Mason
14395202	Christine Bohan
